In [ ]:
using CSV, Tables
using JuMP
using Gurobi

import Pkg; Pkg.add("JSON3")
using JSON3

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [4]:
const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-08


## load in data

In [5]:
all_classes = open("data/all_classes_info.json", "r") do file 
    JSON3.read(file)
end

R = CSV.File("data/ratings.csv",header=0) |> Tables.matrix;
H = CSV.File("data/hours.csv",header=0) |> Tables.matrix;
U = CSV.File("data/units.csv",header=0) |> Tables.matrix;

vars = CSV.File("data/variables.csv",header=0) |> Tables.matrix;
n = vars[1];
S = vars[2];

In [13]:
all_classes[1]

JSON3.Object{Base.CodeUnits{UInt8, String}, SubArray{UInt64, 1, Vector{UInt64}, Tuple{UnitRange{Int64}}, true}} with 28 entries:
  :rating              => 5.4
  :gir_attribute       => "REST"
  :has_final           => false
  :description         => "Presents engineering problems in a computational set…
  :offered_fall        => false
  :offered_spring      => true
  :meets_with_subjects => ["1.001"]
  :instructors         => ["J. Williams"]
  :out_of_class_hours  => 7.99
  :total_units         => 12
  :related_subjects    => ["2.156", "1.205", "1.C51", "1.000", "1.631", "1.063"…
  :pdf_option          => false
  :in_class_hours      => 5.7
  :is_half_class       => false
  :level               => "U"
  :prerequisites       => "GIR:CAL1"
  :subject_id          => "1.00"
  :title               => "Engineering Computation and Data Science"
  :lab_units           => 2
  :design_units        => 0
  :public              => true
  :offered_summer      => false
  :lecture_units       => 3
  :

In [16]:
findfirst(x -> x[:subject_id] == "8.01", all_classes)

3856

In [59]:
chem_gir_ids = ["3.091", "5.111", "5.112"]
phys_1_gir_ids = ["8.01", "8.011", "8.012", "8.01L"]
phys_2_gir_ids = ["8.02", "8.021", "8.022"]
math_1_gir_ids = ["18.01", "18.01A"]
math_2_gir_ids = ["18.02", "18.02A", "18.022"]
bio_gir_ids = ["7.012", "7.013", "7.014", "7.015", "7.016"]
gir_ids = [chem_gir_ids, phys_1_gir_ids, phys_2_gir_ids, math_1_gir_ids, math_2_gir_ids, bio_gir_ids]
gir_subject_types = [[findfirst(x -> x[:subject_id] == gir_id, all_classes) for gir_id in gir_type] for gir_type in gir_ids]

6-element Vector{Vector{Int64}}:
 [2962, 3303, 3304]
 [3856, 3857, 3858, 3859]
 [3860, 3861, 3862]
 [1536, 1537]
 [1538, 1540, 1539]
 [3752, 3753, 3754, 3755, 3756]

## optimize

In [ ]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV))

α = 0.8

# @variable(model, X[1:n,1:s] >=0)
@variable(model, X[1:n, 1:S], Bin)
@objective(model, Min, sum(α*H[i]*X[i,s] - (1-α)*R[i]*X[i,s] for i=1:n,s=1:S));

##
# CONSTRAINTS
##

# full-time; at least 36 units per semester
@constraint(model, [s in 1:S], sum(U[i]*X[i, s] for i=1:n) >= 36);

# wellbeing: no more than 112 hours per week
@constraint(model, [s in 1:S], sum(H[i]*X[i, s] for i=1:n) <= 112);

# not repeatable: cannot take a class again
@constraint(model, [i in 1:n], sum(X[i, t] for t=1:S) <= 1);

# GIR constraints
@constraint(model, [gir_type in gir_subject_types], sum(X[gir_ind, s] for gir_ind in gir_type, s in 1:S) == 1);

# taken correct semester
@constraint(model, [s in 1:2:S, i in 1:n], X[i, s] <= all_classes[i][:offered_fall]);
@constraint(model, [s in 2:2:S, i in 1:n], X[i, s] <= all_classes[i][:offered_spring]);

# 2 CI-H or HW
@constraint(model, sum(
    (get(all_classes[i], :communication_requirement, "") in ["CI-H", "CI-HW"]) * X[i, s] for i=1:n, s=1:S) >= 2);

# 8 HASS
# @constraint(model, sum(
#     (get(all_classes[i], :hass_attribute, "") in ["HASS-H", "HASS-A", "HASS-S"]) * X[i, s] for i=1:n, s=1:S) >= 8);

# 1 of each HASS-H, A, S
@constraint(model, [hass_type in ["HASS-H", "HASS-A", "HASS-S"]], sum(
    (get(all_classes[i], :hass_attribute, "") in [hass_type]) * X[i, s] for i=1:n, s=1:S) >= 1);


In [107]:
optimize!(model)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 42353 rows, 37624 columns and 155976 nonzeros
Model fingerprint: 0x0d9dfab2
Variable types: 0 continuous, 37624 integer (37624 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [2e-02, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 182.6560000
Presolve removed 38267 rows and 19800 columns
Presolve time: 0.18s
Presolved: 4086 rows, 17824 columns, 58692 nonzeros
Found heuristic solution: objective 131.6860000
Variable types: 0 continuous, 17824 integer (17824 binary)
Found heuristic solution: objective 120.0340000

Root relaxation: objective 6.405400e+01, 114 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds 

In [108]:
taken_classes_sem = [[ind for (ind, val) in enumerate(sem) if val == 1] for sem in eachcol(value.(X))]

8-element Vector{Vector{Int64}}:
 [294, 1953, 2021, 2885, 3017, 3755, 4511]
 [144, 714, 1666]
 [1536, 3861, 4492]
 [1350, 3337, 4295, 4347]
 [1180, 1539, 2600, 4160]
 [100, 974, 978, 3857]
 [296, 1324, 1946, 2115, 4141]
 [816, 1025, 2919, 2962]

## Interpreter

In [100]:
function interpret_course_nums(classes_id_per_sem)
    course_nums = [[all_classes[class_ind][:subject_id] for class_ind in sem] for sem in classes_id_per_sem]
    return course_nums
end

function interpret_course_attr(classes_id_per_sem, attr)
    courses_attr = [[get(all_classes[class_ind], Symbol(attr), "") for class_ind in sem] for sem in classes_id_per_sem]
    return courses_attr
end

interpret_course_attr (generic function with 1 method)

In [109]:
interpret_course_nums(taken_classes_sem)

8-element Vector{Vector{String}}:
 ["10.960", "20.053", "20.S900", "24.93", "3.903", "7.015", "MS.101"]
 ["1.713", "12.834", "18.727"]
 ["18.01", "8.021", "MAS.921"]
 ["16.84", "5.561", "ES.113", "HST.533"]
 ["15.839", "18.022", "21M.622", "CMS.827"]
 ["1.267", "15.323", "15.335", "8.011"]
 ["10.991", "16.634", "20.001", "21G.053", "CMS.627"]
 ["14.399", "15.3941", "3.006", "3.091"]

In [111]:
interpret_course_attr(taken_classes_sem, "hass_attribute")

8-element Vector{Vector{String}}:
 ["", "", "", "", "", "", ""]
 ["", "", ""]
 ["", "", ""]
 ["", "", "HASS-H", ""]
 ["", "", "HASS-A", ""]
 ["", "", "", ""]
 ["", "", "", "HASS-S", "HASS-A"]
 ["", "", "", ""]